<a href="https://colab.research.google.com/github/Gooogr/Brain2Image/blob/master/sub_notebooks/EEG_classifier_custom_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EEG signals classifier based on updated 3 paths Perceive Lab dataset<br>
[Code](https://studentiunict-my.sharepoint.com/personal/concetto_spampinato_unict_it/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fconcetto%5Fspampinato%5Funict%5Fit%2FDocuments%2FCode%2Fonline%5Fcode%2Feeg%5Fclassification%5Fcode&originalPath=aHR0cHM6Ly9zdHVkZW50aXVuaWN0LW15LnNoYXJlcG9pbnQuY29tLzpmOi9nL3BlcnNvbmFsL2NvbmNldHRvX3NwYW1waW5hdG9fdW5pY3RfaXQvRW1TMmNVeUVXbHhOcXo5bnBpQmlwSWdCNEdnVVpMcFFJVkQ1cW9pbXBybV9YUT9ydGltZT01dXVDOThwMjJFZw)


In [11]:
## Imports
import sys
import os
import random
import math
import matplotlib.pyplot as plt
import time
from scipy.fftpack import fft, rfft, fftfreq, irfft, ifft, rfftfreq
from scipy import signal
import scipy.io
import numpy as np
import importlib


import torch; torch.utils.backcompat.broadcast_warning.enabled = True
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.backends.cudnn as cudnn; cudnn.benchmark = True
from torchvision import transforms, datasets
from torch.autograd import Variable

#import models
# from layers import * 

In [8]:
! nvidia-smi

Fri Oct 23 22:49:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
! ln -s "/content/drive/My Drive" "/content/mydrive"

Mounted at /content/drive


Old constants

In [ ]:
# DATASET_PATH = '/content/mydrive/EEG2Image_research/datasets/perceive_lab/eeg_signals_128_sequential_band_all_with_mean_std.pth'
# SPLIT_PATH = '/content/mydrive/EEG2Image_research/datasets/perceive_lab/block_splits_by_image.pth'
# SAVE_PATH = '/content/mydrive/EEG2Image_research/models/vanila_lstm_classifier.pth'
# CHKPT_FOLDER = '/content/mydrive/EEG2Image_research/models/chkpt/'

# FILTERING = True       # Default - True, filter raw data
# CHECKPOINT_NAME = None # Default - None, specify checkpoint path for model training

# SPLIT_NUM = 0          # Default - 0, split number
# BATCH_SIZE = 16        # Default - 16, batch size
# NO_CUDA = False        # Default - False, disable CUDA computation

# EPOCHS = 300           # Default - 300, traning epochs number

# LSTM_SIZE = 128        # Default - 128, LSTM hidden size
# LSTM_LAYERS = 1        # Default - 1,  LSTM layers 
# EMBEDDING_SIZE = 128   # Default - 128, embedding size
# NUM_CLASSES = 40       # Default - 40, amount of datasets classses
# OPTIM_NAME = 'Adam'    # Default - Adam, optimizer type
# LR = 0.001             # Default - 0.001, learning rate 

# LR_DECAY_FACTOR = 0.5  # Default - 0.5, learning rate decay factor, for SGD only
# LR_DECAY_PERIOD = 10   # Default - 10  learning rate decay period, for SGD only

### Constants

In [14]:
# -----Dataset options-----
## --eeg-dataset
EEG_DATASET = '/content/mydrive/EEG2Image_research/datasets/perceice_lab_new/eeg_55_95_std.pth'
#EEG_DATASET = '/content/mydrive/EEG2Image_research/datasets/perceice_lab_new/eeg_5_95_std.pth'
#EEG_DATASET = '/content/mydrive/EEG2Image_research/datasets/perceice_lab_new/eeg_14_70_std.pth'

## --splits-path
SPLIT_PATH = '/content/mydrive/EEG2Image_research/datasets/perceice_lab_new/block_splits_by_image_all.pth'
#SPLIT_PATH = '/content/mydrive/EEG2Image_research/datasets/perceice_lab_new/block_splits_by_image_single.pth'

## --split-num
SPLIT_NUM = 0 # Default - 0, type=int, leave this always to zero.
## --subject
SUBJECT = 0 # Default - 0, type=int, range = [1..6] or 0 - all subjects

# -----Time options-----
##--time_low
TIME_LOW = 20.0 # Default - 20.0, type=float, lowest time value
##--time_high
TIME_HIGH = 460.0 # Default - 460.0, type=float, highest time value

# -----Model options-----
##--model_type
MODEL_TYPE = 'lstm' # Default - 'syncnet', range = ['lstm', 'EEGNet', 'syncnet']
# - lstm is the model described in the paper "Deep Learning Human Mind for Automated Visual Classification”, in CVPR 2017
# - EEGNet is the model described in the paper "EEGNet: a compact convolutional neural network for EEG-based brain–computer interfaces", Journal of Neural Engineering 2018
# - syncnet is the model described in the paper "Targeting EEG/LFP synchrony with neural nets", NIPS 2017

## ????
#parser.add_argument('-mp','--model_params', default='', nargs='*', help='list of key=value pairs of model options')
#parser.add_argument('--pretrained_net', default='', help="path to pre-trained net (to continue training)")

#-----Training options-----
##--batch_size
BATCH_SIZE = 16 # Default - 16
##--optim
OPTIM = 'Adam' # Default - 'Adam'
##--learning-rate
LR = 0.001 # Default - 0.001
##--learning-rate-decay-by
LRDB = 0.5 # Default - 0.5, learning rate decay factor
##--learning-rate-decay-every
LRDE = 10 # Default - 10, learning rate decay period
##--data-workers
WORKERS = 4 # Default - 4
##--epochs
EPOCHS = 200 # Default - 200



#-----Save options-----
##--saveCheck
SAVE_CHECK = 100 # Default - 100, Checkpoint period

#-----Backend options-----
NO_CUDA = False



### Dataset generator

In [19]:
# Dataset class
class EEGDataset:
    
    # Constructor
    def __init__(self, eeg_signals_path):
        # Load EEG signals
        loaded = torch.load(eeg_signals_path)
        if SUBJECT != 0:
            self.data = [loaded['dataset'][i] for i in range(len(loaded['dataset']) ) if loaded['dataset'][i]['subject']==SUBJECT]
        else:
            self.data=loaded['dataset']        
        self.labels = loaded["labels"]
        self.images = loaded["images"]
        
        # Compute size
        self.size = len(self.data)

    # Get size
    def __len__(self):
        return self.size

    # Get item
    def __getitem__(self, i):
        # Process EEG
        eeg = self.data[i]["eeg"].float().t()
        eeg = eeg[TIME_LOW:TIME_HIGH,:]

        # Get label
        label = self.data[i]["label"]
        # Return
        return eeg, label

# Splitter class
class Splitter:

    def __init__(self, dataset, split_path, split_num=0, split_name="train"):
        # Set EEG dataset
        self.dataset = dataset
        # Load split
        loaded = torch.load(split_path)
        self.split_idx = loaded["splits"][split_num][split_name]
        # Filter data
        self.split_idx = [i for i in self.split_idx if 450 <= self.dataset.data[i]["eeg"].size(1) <= 600]
        # Compute size
        self.size = len(self.split_idx)

    # Get size
    def __len__(self):
        return self.size

    # Get item
    def __getitem__(self, i):
        # Get sample from dataset
        eeg, label = self.dataset[self.split_idx[i]]
        # Return
        return eeg, label

## Models Zoo

### Base layers

In [22]:
class ConvLayer2D(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel, stride, padding, dilation):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(in_channels))
        self.add_module('relu', nn.ReLU(True))
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=kernel,
                                          stride=stride, padding=padding, dilation=dilation, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        return super().forward(x)

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, n_layers, kernel_size, stride, dilation_list, in_size):
        super().__init__()
        if len(dilation_list) < n_layers:
            dilation_list = dilation_list + [dilation_list[-1]] * (n_layers - len(dilation_list))

        padding = []
        # Compute padding for each temporal layer to have a fixed size output
        # Output size is controlled by striding to be 1 / 'striding' of the original size
        for dilation in dilation_list:
            filter_size = kernel_size[1] * dilation[1] - 1
            temp_pad = math.floor((filter_size - 1) / 2) - 1 * (dilation[1] // 2 - 1)
            padding.append((0, temp_pad))

        self.layers = nn.ModuleList([
            ConvLayer2D(
                in_channels, out_channels, kernel_size, stride, padding[i], dilation_list[i]
            ) for i in range(n_layers)
        ])

    def forward(self, x):
        features = []

        for layer in self.layers:
            out = layer(x)
            features.append(out)

        out = torch.cat(features, 1)
        return out

class SpatialBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_spatial_layers, stride, input_height):
        super().__init__()
       
        kernel_list = []
        for i in range(num_spatial_layers):
            kernel_list.append(((input_height // (i + 1)), 1))

        padding = []
        for kernel in kernel_list:
            temp_pad = math.floor((kernel[0] - 1) / 2)# - 1 * (kernel[1] // 2 - 1)
            padding.append((temp_pad, 0))

        feature_height = input_height // stride[0]

        self.layers = nn.ModuleList([
            ConvLayer2D(
                in_channels, out_channels, kernel_list[i], stride, padding[i], 1
            ) for i in range(num_spatial_layers)
        ])
    
    def forward(self, x):
        features = []

        for layer in self.layers:
            out = layer(x)
            features.append(out)

        out = torch.cat(features, 1)

        return out

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [23]:
# EEGChannelNet

class FeaturesExtractor(nn.Module):
    def __init__(self, in_channels, temp_channels, out_channels, input_width, in_height,
                 temporal_kernel, temporal_stride, temporal_dilation_list, num_temporal_layers,
                 num_spatial_layers, spatial_stride, num_residual_blocks, down_kernel, down_stride):
        super().__init__()

        self.temporal_block = TemporalBlock(
            in_channels, temp_channels, num_temporal_layers, temporal_kernel, temporal_stride, temporal_dilation_list, input_width
        )

        self.spatial_block = SpatialBlock(
            temp_channels * num_temporal_layers, out_channels, num_spatial_layers, spatial_stride, in_height
        )

        self.res_blocks = nn.ModuleList([
            nn.Sequential(
                ResidualBlock(
                    out_channels * num_spatial_layers, out_channels * num_spatial_layers
                ),
                ConvLayer2D(
                    out_channels * num_spatial_layers, out_channels * num_spatial_layers, down_kernel, down_stride, 0, 1
                )
            ) for i in range(num_residual_blocks)
        ])

        self.final_conv = ConvLayer2D(
            out_channels * num_spatial_layers, out_channels, down_kernel, 1, 0, 1
        )

    def forward(self, x):
        out = self.temporal_block(x)

        out = self.spatial_block(out)

        if len(self.res_blocks) > 0:
            for res_block in self.res_blocks:
                out = res_block(out)

        out = self.final_conv(out)
        
        return out

class Model(nn.Module):
    '''The model for EEG classification.
    The imput is a tensor where each row is a channel the recorded signal and each colums is a time sample.
    The model performs different 2D to extract temporal e spatial information.
    The output is a vector of classes where the maximum value is the predicted class.
    Args:
        in_channels: number of input channels
        temp_channels: number of features of temporal block
        out_channels: number of features before classification
        num_classes: number possible classes
        embedding_size: size of the embedding vector
        input_width: width of the input tensor (necessary to compute classifier input size)
        input_height: height of the input tensor (necessary to compute classifier input size)
        temporal_dilation_list: list of dilations for temporal convolutions, second term must be even
        temporal_kernel: size of the temporal kernel, second term must be even (default: (1, 32))
        temporal_stride: size of the temporal stride, control temporal output size (default: (1, 2))
        num_temp_layers: number of temporal block layers
        num_spatial_layers: number of spatial layers
        spatial_stride: size of the spatial stride
        num_residual_blocks: the number of residual blocks
        down_kernel: size of the bottleneck kernel
        down_stride: size of the bottleneck stride
        '''
    def __init__(self, in_channels=1, temp_channels=10, out_channels=50, num_classes=40, embedding_size=1000,
                 input_width=440, input_height=128, temporal_dilation_list=[(1,1),(1,2),(1,4),(1,8),(1,16)],
                 temporal_kernel=(1,33), temporal_stride=(1,2),
                 num_temp_layers=4,
                 num_spatial_layers=4, spatial_stride=(2,1), num_residual_blocks=4, down_kernel=3, down_stride=2):
        super().__init__()

        self.encoder = FeaturesExtractor(in_channels, temp_channels, out_channels, input_width, input_height,
                                     temporal_kernel, temporal_stride,
                                     temporal_dilation_list, num_temp_layers,
                                     num_spatial_layers, spatial_stride, num_residual_blocks, down_kernel, down_stride
                                     )

        encoding_size = self.encoder(torch.zeros(1, in_channels, input_height, input_width)).contiguous().view(-1).size()[0]

        self.classifier = nn.Sequential(
            nn.Linear(encoding_size, embedding_size),
            nn.ReLU(True),
            nn.Linear(embedding_size, num_classes), 
        )

    def forward(self, x):
        out = self.encoder(x)

        out = out.view(x.size(0), -1)

        out = self.classifier(out)

        return out

### LSTM

In [ ]:
class Model(nn.Module):

    def __init__(self, input_size=128, lstm_size=128, lstm_layers=1, output_size=128):
        # Call parent
        super().__init__()
        # Define parameters
        self.input_size = input_size
        self.lstm_size = lstm_size
        self.lstm_layers = lstm_layers
        self.output_size = output_size

        # Define internal modules
        self.lstm = nn.LSTM(input_size, lstm_size, num_layers=lstm_layers, batch_first=True)
        self.output = nn.Linear(lstm_size, output_size)
        self.classifier = nn.Linear(output_size,40)
        
    def forward(self, x):
        # Prepare LSTM initiale state
        batch_size = x.size(0)
        lstm_init = (torch.zeros(self.lstm_layers, batch_size, self.lstm_size), torch.zeros(self.lstm_layers, batch_size, self.lstm_size))
        if x.is_cuda: lstm_init = (lstm_init[0].cuda(), lstm_init[0].cuda())
        lstm_init = (Variable(lstm_init[0], volatile=x.volatile), Variable(lstm_init[1], volatile=x.volatile))

        # Forward LSTM and get final state
        x = self.lstm(x, lstm_init)[0][:,-1,:]
        
        # Forward output
        x = F.relu(self.output(x))
        x = self.classifier((x))
        return x

### EEGNet

In [ ]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.T = 440
        self.channels = 128
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 32), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(97, 32, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(32, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(32, 64, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(64, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # Layer 4
        self.padding3 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv4 = nn.Conv2d(64, 128, (8, 4))
        self.batchnorm4 = nn.BatchNorm2d(128, False)
        self.pooling4 = nn.MaxPool2d((2, 6))
        

        # Layer 5
        self.padding4 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv5 = nn.Conv2d(128, 256, (8, 4))
        self.batchnorm5 = nn.BatchNorm2d(256, False)
        self.pooling5 = nn.MaxPool2d((2, 4))
        
        # Layer 6
        self.padding5 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv6 = nn.Conv2d(256, 512, (8, 4))
        self.batchnorm6 = nn.BatchNorm2d(512, False)
        self.pooling6 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        self.fc1 = nn.Linear(512, 40)
        
    def forward(self, x):
        x = x.view(x.size(0),1,x.size(1),x.size(2))
        x = x.permute(0,1,3,2)
        
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # Layer 4
        x = self.padding3(x)
        x = F.elu(self.conv4(x))
        x = self.batchnorm4(x)
        x = F.dropout(x, 0.25)
        x = self.pooling4(x)

        # Layer 5
        #x = self.padding4(x)
        #x = F.elu(self.conv5(x))
        #x = self.batchnorm5(x)
        #x = F.dropout(x, 0.25)
        #x = self.pooling5(x)

        # Layer 6
        #x = self.padding5(x)
        #x = F.elu(self.conv6(x))
        #x = self.batchnorm6(x)
        #x = F.dropout(x, 0.25)
        #x = self.pooling6(x)

        # FC Layer
        x = x.view(-1, 512)
        x = torch.sigmoid(self.fc1(x))
        return x

### Syncnet

In [ ]:
class Model(nn.Module):
    def __init__(self, num_channels=128, input_size=440, num_classes=40, num_filters=128, filter_width = 40, pool_size=40):
        super(Model, self).__init__()
        self.num_filters = num_filters 
        self.num_channels = num_channels
        self.filter_width = filter_width
        
        self.b = nn.Parameter(torch.FloatTensor(1,1,num_channels,num_filters).uniform_(-0.05, 0.05))
        self.bias = nn.Parameter(torch.FloatTensor(num_filters).fill_(0))
        self.omega = nn.Parameter(torch.FloatTensor(1,1,1,num_filters).uniform_(0, 1))
        self.cl_Wy = int(np.ceil(float(input_size)/float(pool_size)) * num_filters)
        
        if(filter_width%2 == 0):
            self.t = nn.Parameter(torch.FloatTensor(np.reshape(range(-int(filter_width/2),int(filter_width/2)),[1,int(filter_width),1,1])).fill_(0))
        else:
            self.t = nn.Parameter(torch.FloatTensor(np.reshape(range(-int((filter_width-1)/2),int((filter_width-1)/2) + 1),[1,int(filter_width) ,1,1])).fill_(0))
        
        self.phi_ini = nn.Parameter(torch.FloatTensor(1,1,num_channels, num_filters).normal_(0,0.05))
        self.beta = nn.Parameter(torch.FloatTensor(1,1,1,num_filters).uniform_(0, 0.05))
        
        ## Only stride and dilation values of 1 are supported. If you use different values, padding values wont be correct
        P = ((input_size-1)-input_size + (filter_width-1))
        if(P%2 == 0):
            self.padding = (P//2, P//2 + 1)
        else:
            self.padding = (P//2, P//2)
        
        self.pool = nn.MaxPool2d((1, pool_size), stride = (1, pool_size))
        self.classifier = nn.Linear(self.cl_Wy,num_classes)
    
    def forward(self,X):
        #X must be in the form of BxCx1xT or BxCxT
        X = X.permute(0,2,1)

        self.W_osc = torch.mul(self.b,torch.cos(self.t*self.omega + self.phi_ini))
        self.W_decay = torch.exp(-torch.pow(self.t,2)*self.beta)
        self.W = torch.mul(self.W_osc,self.W_decay)
        self.W = self.W.view(self.num_filters,self.num_channels,1,self.filter_width)
        if(len(X.size()) == 3):
            X = X.unsqueeze(2)
    
        res = F.conv2d(F.pad(X,self.padding,"constant", 0).float(),self.W.float(),bias = self.bias,stride=1 )
        res = self.pool(F.pad(res,self.padding, "constant", 0))
        res = res.view(-1,self.cl_Wy)
        self.beta = nn.Parameter(torch.clamp(self.beta, min=0))
        return self.classifier(F.relu(res)).squeeze()

## Main training loop

In [20]:
%%time
# Load dataset
dataset = EEGDataset(EEG_DATASET)
# Create loaders
loaders = {split: DataLoader(Splitter(dataset, 
                                      split_path = SPLIT_PATH, 
                                      split_num = SPLIT_NUM, 
                                      split_name = split), 
                             batch_size = BATCH_SIZE, 
                             drop_last = True, 
                             shuffle = True) for split in ["train", "val", "test"]}

CPU times: user 494 ms, sys: 3.15 s, total: 3.64 s
Wall time: 59.3 s


In [ ]:
##########################################
############################################

# MAIN SCRIPT

In [ ]:
# Load model

model_options = {key: int(value) if value.isdigit() else (float(value) if value[0].isdigit() else value) for (key, value) in [x.split("=") for x in opt.model_params]}
# Create discriminator model/optimizer
module = importlib.import_module("models." + opt.model_type)
model = module.Model(**model_options)
optimizer = getattr(torch.optim, opt.optim)(model.parameters(), lr = opt.learning_rate)
    
# Setup CUDA
if not opt.no_cuda:
    model.cuda()
    print("Copied to CUDA")

if opt.pretrained_net != '':
        model = torch.load(opt.pretrained_net)
        print(model)

#initialize training,validation, test losses and accuracy list
losses_per_epoch={"train":[], "val":[],"test":[]}
accuracies_per_epoch={"train":[],"val":[],"test":[]}

best_accuracy = 0
best_accuracy_val = 0
best_epoch = 0
# Start training

predicted_labels = [] 
correct_labels = []

for epoch in range(1, opt.epochs+1):
    # Initialize loss/accuracy variables
    losses = {"train": 0, "val": 0, "test": 0}
    accuracies = {"train": 0, "val": 0, "test": 0}
    counts = {"train": 0, "val": 0, "test": 0}
    # Adjust learning rate for SGD
    if opt.optim == "SGD":
        lr = opt.learning_rate * (opt.learning_rate_decay_by ** (epoch // opt.learning_rate_decay_every))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    # Process each split
    for split in ("train", "val", "test"):
        # Set network mode
        if split == "train":
            model.train()
            torch.set_grad_enabled(True)
        else:
            model.eval()
            torch.set_grad_enabled(False)
        # Process all split batches
        for i, (input, target) in enumerate(loaders[split]):
            # Check CUDA
            if not opt.no_cuda:
                input = input.to("cuda") 
                target = target.to("cuda") 
            # Forward
            output = model(input)

            # Compute loss
            loss = F.cross_entropy(output, target)
            losses[split] += loss.item()
            # Compute accuracy
            _,pred = output.data.max(1)
            correct = pred.eq(target.data).sum().item()
            accuracy = correct/input.data.size(0)   
            accuracies[split] += accuracy
            counts[split] += 1
            # Backward and optimize
            if split == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    
    # Print info at the end of the epoch
    if accuracies["val"]/counts["val"] >= best_accuracy_val:
        best_accuracy_val = accuracies["val"]/counts["val"]
        best_accuracy = accuracies["test"]/counts["test"]
        best_epoch = epoch
    
    TrL,TrA,VL,VA,TeL,TeA=  losses["train"]/counts["train"],accuracies["train"]/counts["train"],losses["val"]/counts["val"],accuracies["val"]/counts["val"],losses["test"]/counts["test"],accuracies["test"]/counts["test"]
    print("Model: {11} - Subject {12} - Time interval: [{9}-{10}]  [{9}-{10} Hz] - Epoch {0}: TrL={1:.4f}, TrA={2:.4f}, VL={3:.4f}, VA={4:.4f}, TeL={5:.4f}, TeA={6:.4f}, TeA at max VA = {7:.4f} at epoch {8:d}".format(epoch,
                                                                                                         losses["train"]/counts["train"],
                                                                                                         accuracies["train"]/counts["train"],
                                                                                                         losses["val"]/counts["val"],
                                                                                                         accuracies["val"]/counts["val"],
                                                                                                         losses["test"]/counts["test"],
                                                                                                         accuracies["test"]/counts["test"],
                                                                                                         best_accuracy, best_epoch, opt.time_low,opt.time_high, opt.model_type,opt.subject))

    losses_per_epoch['train'].append(TrL)
    losses_per_epoch['val'].append(VL)
    losses_per_epoch['test'].append(TeL)
    accuracies_per_epoch['train'].append(TrA)
    accuracies_per_epoch['val'].append(VA)
    accuracies_per_epoch['test'].append(TeA)

    if epoch%opt.saveCheck == 0:
                torch.save(model, '%s__subject%d_epoch_%d.pth' % (opt.model_type, opt.subject,epoch))
            